In [207]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torchvision.datasets
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

In [208]:
transform = transforms.Compose([
    transforms.Resize(256),             
    transforms.CenterCrop(224),        
    transforms.ToTensor(),             
    transforms.Normalize(              
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

In [209]:
class CSVImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = 'data/images/' + self.df.iloc[idx]
        
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
            
        return image
    def to_tensor(self):
        tensor = []
        for i in range(len(self)):
            img_tensor = self[i]
            tensor.append(img_tensor)
        return torch.stack(tensor)

In [210]:
data = pd.read_csv('data/train.csv')
X = data['image']
y = data['class']

In [211]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1000)

In [212]:
X_train = CSVImageDataset(X_train, transform=transform)
X_train = X_train.to_tensor()
X_test = CSVImageDataset(X_test, transform=transform)
X_test = X_test.to_tensor()
y_train = torch.from_numpy(np.array(y_train)).long()
y_test = torch.from_numpy(np.array(y_test)).long()
y_test

tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [213]:
class RostelecomNet(torch.nn.Module):
    def __init__(self):
        super(RostelecomNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=11, padding=0)
        self.act1 = torch.nn.Tanh()
        self.pool1 = torch.nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = torch.nn.Conv2d(in_channels=2, out_channels=1, kernel_size=11, padding=0)
        self.act2 = torch.nn.Tanh()
        self.pool2 = torch.nn.AvgPool2d(kernel_size=4, stride=4)
        self.conv3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=5, padding=0)
        self.act3 = torch.nn.Tanh()
        self.pool3 = torch.nn.AvgPool2d(kernel_size=3, stride=3)
        self.fc1 = torch.nn.Linear(1 * 6 * 6, 2)
        self.sm = torch.nn.Softmax()
    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.act3(x)
        x = self.pool3(x)
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        return x
    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
rostelecomnet = RostelecomNet()

In [214]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rostelecomnet.parameters(), lr=1.0e-2)

In [215]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
rostelecomnet = rostelecomnet.to(device)

In [216]:
batch_size = 100 # Размер батча

for epoch in range(5000): # Кол-во эпох 5000
    # Перемешиваем данные
    order = np.random.permutation(len(X_train))
    # Итерируемся по батчам
    for start_index in range(0, len(X_train), batch_size):
        # Обнуляем градиенты
        optimizer.zero_grad()
        # Индексы элементов батча
        batch_indexes = order[start_index:start_index+batch_size]
        # Элементы батча
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        # Предсказания
        preds = rostelecomnet.forward(x_batch)
        # Значение функции потерь
        loss_value = loss(preds, y_batch)
        # Получаем градиенты
        loss_value.backward()
        # Шаг градиентного спуска
        optimizer.step()
    # Каждые 100 эпох делаем предсказания для тестовой выборки
    if epoch % 10 == 0:
        # Сами предсказания
        test_preds = rostelecomnet.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print(f1_score(test_preds, y_test)) #Получаем точность по метрике f1-score(macro)
        if f1_score(test_preds, y_test) >= 0.95:
            break
        

0.8961038961038961
0.9324324324324325
0.9517241379310345


In [217]:
def predict(net, x):
    y_pred = net.inference(x)
    y_pred = y_pred.argmax(dim=1)
    return y_pred

In [218]:
test_data = pd.read_csv('data/test.csv')
images = test_data['image']
test = CSVImageDataset(images, transform=transform)
test = test.to_tensor()

In [219]:
y_preds = predict(rostelecomnet, test)

test_data['class'] = y_preds.detach().numpy()
test_data.to_csv('data/submit.csv')
test_data

c:\Users\Пользователь\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


,image,class
0,pnlioc.jpg,1
1,hivmnw.jpg,1
2,fsftlx.jpg,1
3,uygvah.jpg,0
4,kmcfhr.jpg,1
...,...,...
156,dovvvc.jpg,1
157,axrglz.jpg,1
158,xyxrkm.jpg,0
159,xosnep.jpg,1
